In [1]:
import torch
import pandas as pd
import re
import os
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer(language='russian')
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_csv(r"C:\Docs\GitHub\NLP_Project_SchoolX\dataset\dataset.csv", names = ['Review', 'Rating'], sep=',')
df = df.dropna()
np.random.seed(3467)
df1 = df.sample(frac = 0.5)

In [3]:
df1 = df1.applymap(lambda s:s.lower() if type(s) == str else s)

In [4]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [5]:
df1["no_punkt"] = df['Review'].str.replace('[^\w\s]','')

C:\Users\coolz\AppData\Local\Temp/ipykernel_5892/1529920971.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df1["no_punkt"] = df['Review'].str.replace('[^\w\s]','')


In [6]:
print (df.columns.tolist())

['Review', 'Rating']


In [7]:
stop_words = stopwords.words('russian')
df1['clean'] = df1['no_punkt'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [8]:
df1['tokenized_reviews'] = df1.apply(lambda row: nltk.word_tokenize(row['clean']), axis=1)

In [9]:
df1['stemmed'] = df1.apply(lambda row: [stemmer.stem(item) for item in row.clean], axis=1)

In [10]:
X = df1['tokenized_reviews']
y = df1['Rating']

In [11]:
df1

,Review,Rating,no_punkt,clean,tokenized_reviews,stemmed
1411,"htc разочаровали, не хотят делать качественные...",neg,htc разочаровали не хотят делать качественные ...,htc разочаровали хотят делать качественные мод...,"[htc, разочаровали, хотят, делать, качественны...","[h, t, c, , р, а, з, о, ч, а, р, о, в, а, л, ..."
877,в целом аппарат очень достойный. проблем и глю...,pos,в целом аппарат очень достойный проблем и глюк...,целом аппарат очень достойный проблем глюков з...,"[целом, аппарат, очень, достойный, проблем, гл...","[ц, е, л, о, м, , а, п, п, а, р, а, т, , о, ..."
1874,"был у меня несколько дней на тест-драйв, т.к. ...",pos,был у меня несколько дней на тестдрайв тк прис...,несколько дней тестдрайв тк присматриваю замен...,"[несколько, дней, тестдрайв, тк, присматриваю,...","[н, е, с, к, о, л, , к, о, , д, н, е, й, , т..."
555,"итак, после недели пользования аппаратом довол...",pos,итак после недели пользования аппаратом доволе...,итак недели пользования аппаратом доволен задн...,"[итак, недели, пользования, аппаратом, доволен...","[и, т, а, к, , н, е, д, е, л, и, , п, о, л, ..."
1581,телефон брать не рекомендую и это ещё мягко ск...,neg,телефон брать не рекомендую и это ещё мягко ск...,телефон брать рекомендую это ещё мягко сказано,"[телефон, брать, рекомендую, это, ещё, мягко, ...","[т, е, л, е, ф, о, н, , б, р, а, т, , , р, е..."
1175,"пользуюсь телефоном несколько дней, поэтому от...",pos,пользуюсь телефоном несколько дней поэтому отз...,пользуюсь телефоном несколько дней поэтому отз...,"[пользуюсь, телефоном, несколько, дней, поэтом...","[п, о, л, , з, у, ю, с, , , т, е, л, е, ф, о,..."
1159,"хороший телефон, удобный в использовании, за п...",pos,хороший телефон удобный в использовании за пол...,хороший телефон удобный использовании полгода ...,"[хороший, телефон, удобный, использовании, пол...","[х, о, р, о, ш, и, й, , т, е, л, е, ф, о, н, ..."
859,телефоном пользуюсь почти год. первый аппарат ...,neg,телефоном пользуюсь почти год первый аппарат о...,телефоном пользуюсь год первый аппарат lg поку...,"[телефоном, пользуюсь, год, первый, аппарат, l...","[т, е, л, е, ф, о, н, о, м, , п, о, л, , з, у..."
1369,"встроенная память 8 гб, мало, но есть разъём п...",pos,встроенная память 8 гб мало но есть разъём под...,встроенная память 8 гб мало разъём карту памят...,"[встроенная, память, 8, гб, мало, разъём, карт...","[в, с, т, р, о, е, н, н, а, я, , п, а, м, я, ..."
602,"в принципе аппарат близок к идеалу, но все пор...",pos,в принципе аппарат близок к идеалу но все порт...,принципе аппарат близок идеалу портят лаги,"[принципе, аппарат, близок, идеалу, портят, лаги]","[п, р, и, н, ц, и, п, е, , а, п, п, а, р, а, ..."


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                        test_size = 0.5, random_state=24)
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
#Vectorizing the text data
ctmTr = cv.fit_transform(X_train)
X_test_dtm = cv.transform(X_test)
from sklearn.linear_model import LogisticRegression
#Training the model
lr = LogisticRegression()
lr.fit(ctmTr, y_train)
#Accuracy score
lr_score = lr.score(X_test_dtm, y_test)
print("Accuracy")
print(lr_score)
#Predicting the labels for test data
y_pred_lr = lr.predict(X_test_dtm)
from sklearn.metrics import confusion_matrix
#Confusion matrix
cm_lr = confusion_matrix(y_test, y_pred_lr)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_lr).ravel()
print(tn, fp, fn, tp)
#True positive and true negative rates
tpr_lr = round(tp/(tp + fn), 4)
tnr_lr = round(tn/(tn+fp), 4)
print(tpr_lr, tnr_lr)

AttributeError: 'list' object has no attribute 'lower'